#### IMPORTAMOS LIBRERIAS

In [49]:
from optparse import Option
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import random
from time import sleep
import pandas as pd
import numpy as np

#### FUNCION PARA OBTENER CREDENCIALES

In [50]:
def get_credentials():
    credentials = open('credentials.txt').readline().strip().split(',')
    user = credentials[0]
    password = credentials[1]
    
    return user, password

#### CREAMOS EL DRIVER

In [51]:

def get_driver():
    options = Options()
    #options.add_argument(
    #    "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
    #)
    options.add_argument("--disable-notifications")
    options.add_argument("--headles")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-software-rasterizer")
    options.add_argument('--log-level=3')

    options.add_experimental_option('excludeSwitches', ['enable-logging'])


    driver = webdriver.Chrome('./chromedriver.exe', options = options)
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    
    
    sleep(random.uniform(3,4)) 

    return driver

#### FUNCIONES PRINCIPALES PARA SCRAPEAR

In [52]:
def cleaning_text(text):
    clean_text = text.replace('\n', ' ').replace('\r', ' ').replace('\t',' ').replace(',','').replace('~','').strip()
    return clean_text


def logging_instagram(driver):
    username, password = get_credentials()
    username_input = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@name="username"]'))
        )
    password_input =  WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@name="password"]'))
        )
    username_input.send_keys(username)
    password_input.send_keys(password)

    sleep(random.uniform(2,3)) 

    button = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//button[@type="submit"]'))
        )
    button.click()

    sleep(random.uniform(3,4))


def search_instagram(driver, HASHTAG):
    search_input =  WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@placeholder="Busca"]'))
        )
    search_input.send_keys(HASHTAG)

    sleep(random.uniform(3,4))

    search_link = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, f'//a[@href="/explore/tags/{HASHTAG.replace("#", "")}/"]'))
        )
    search_link.click()

    sleep(random.uniform(4,5))


def scraping_instagram(driver):

    data = []

    # Scrolling
    for i in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(random.uniform(4,5))
    
    posts = driver.find_elements(By.XPATH, '//article//a')
    for post in posts:
        try:
            post.click()      
            
            user = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//article[@role="presentation"]//h2//a'))).text
            description = cleaning_text(driver.find_element(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span').text)
            
            try:
                hashtags = driver.find_elements(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span/a[@class != "notranslate"]')
                hashtags = [cleaning_text(h.text) for h in hashtags]
            except:
                hashtags = []
            
            try:
                labels = driver.find_elements(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span/a[@class="notranslate"]')
                labels = [cleaning_text(l.text) for l in labels]
            except: 
                labels= []

            try:
                likes = driver.find_element(By.XPATH, '//article[@role="presentation"]//a[@class="zV_Nj"]/span').text
            except:
                likes = 0

            data.append({
                "user": user,
                "likes": likes,
                "labels":labels,
                "hashtag":hashtags,
                "description": description
            })

            # We use driver here, because post is limited to their scope 'a'
            close_button = WebDriverWait(driver, 10).until(   
                EC.presence_of_element_located((By.XPATH, '//div[@class="NOTWr"]/button'))
            )
            sleep(random.uniform(1,2))
            close_button.click()

        except Exception as e:
            continue
        
    df = pd.DataFrame(data)
    return df

def save_dataframe_to_csv(df):
    df.to_csv(r'data.csv', header=True, index=True, sep='|')

#### FUNCION TRIGGER QUE LANZA EL PROCESO

In [53]:
def run_scraping_process():
    HASHTAG = '#pikachu'
    driver = get_driver()
    logging_instagram(driver)
    search_instagram(driver, HASHTAG)
    df = scraping_instagram(driver)
    save_dataframe_to_csv(df)
    return df

In [54]:
data = run_scraping_process()
data

C:\Users\ggress\AppData\Local\Temp\ipykernel_21396\2026090804.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', options = options)


,user,likes,labels,hashtag,description
0,anime_3.0ads,215,[@anime_3.0ads],"[#animesbrasil, #animes, #pain, #minatouzumaki...",✏️Aprenda Desenhar seu anime com o método fana...
1,neongamingartist,211,[],"[#pikachu, #pikachu⚡, #pikapika, #pikachumemes...",It's Pikasaur! 😱 Or is it Bulbachu? 🤔 Oh it's...
2,pogotrnr,191,[],"[#GOsnapshot, #pokemon, #pokemongo, #pokemongo...",A party of Plusle amused by the flowers⚡️➕🔴 • ...
3,desenhosvirall,626,"[@desenhosvirall, @jessyca_draw]","[#desenho, #desenhos, #animearts, #animesketch...",Dê dois cliques na tela se você gostou! 🔥🖌️ 📌...
4,nanaririnari,5.364,[],"[#生チョコタルト, #ピカチュウ, #おしり, #ミニタルト, #キャラフード, #生チョ...",#生チョコタルト 市販のミニタルトでクマ型があってっ(可愛い♥️)♥️ ひっくり返したらおし...
5,keisuke7g,346,[@skymark_jpn],"[#skymarkairlines, #skymark, #スカイマーク, #boeing,...",SKYMARK AIRLINES 🇯🇵 B737-8AL(WL) ✈️ JA73AB . 撮...
6,pikachan_pya,354,[],"[#ぬい撮り, #ぬいぐるみごはん, #ぴかりん, #pikachu, #皮卡丘, #コスト...",ぴかりん「ピザとソフトクリーム買ってきたぴか🤗安ウマぴか😋」 #ぬい撮り #ぬいぐるみごはん...
